# RAB Dashboarding Notebook

In [26]:
from box import Box
import pandas as pd
import altair as alt
import rab
import zoho

In [5]:
# Load the config settings
with open("config.yml", "r") as config_file:
    config = Box.from_yaml(config_file)

In [14]:
crm = zoho.Crm(
    org_id = config.zoho.org_id, 
    client_id = config.zoho.client_id, 
    client_secret = config.zoho.client_secret, 
    redirect = config.zoho.redirect, 
    scope = config.zoho.scope.to_list()
)

127.0.0.1 - - [11/Feb/2021 15:11:52] "GET /?code=1000.f807610e034508498b897974bd8fd652.d7737003f0dcd7225e9566bb54833f94&location=us&accounts-server=https%3A%2F%2Faccounts.zoho.com& HTTP/1.1" 200 -


In [19]:
r = crm.get("Invoices")

In [16]:
air_br =  rab.Rab()

Getting the Registro Aeronáutico Brasileiro (RAB) from Brazilian ANAC
Download Successful
Cleaning the Raw Registro Aeronáutico Brasileiro (RAB)


In [17]:
air_br.clean.head()

,tail_number,owner_customer_name,owner_other,owner_state,owner_tax_id,operator_customer_name,operator_other,operator_state,operator_tax_id,certifate_num,...,unk_cdmarcanac3,foreign_tail_number,unk_dsgravame,age,owner_tax_id_type,owner_tax_id_print,operator_tax_id_type,operator_tax_id_print,owned_operated,agaircraft
0,PRYYX,AZUL LINHAS AEREAS BRASILEIRAS S.A.,NaN,SP,09296295000160,NaN,NaN,NaN,<NA>,NaN,...,NaN,NaN,RESERVADAS AS MARCAS,<NA>,CNPJ,09.296.295/0001-60,EMPTY,<NA>,False,False
1,PRZAB,AMT PATRIMONIAL E PARTCIPACOES LTDA,FERNANDO SOARES GURGEL,CE,29327303000103,AMT PATRIMONIAL E PARTCIPACOES LTDA,FERNANDO SOARES GURGEL,CE,29327303000103,121229,...,NaN,NaN,NaN,9,CNPJ,29.327.303/0001-03,CNPJ,29.327.303/0001-03,True,False
2,PRZAD,AIRSHIP DO BRASIL - INDUSTRIA E SERVICOS AEREO...,NaN,SP,07933461000166,AIRSHIP DO BRASIL - INDUSTRIA E SERVICOS AEREO...,NaN,SP,07933461000166,201106,...,NaN,NaN,NaN,1,CNPJ,07.933.461/0001-66,CNPJ,07.933.461/0001-66,True,False
3,PRZAE,ADEMIR BATISTA BRAGA,NaN,SP,84293918868,ADEMIR BATISTA BRAGA,NaN,SP,84293918868,110921,...,NaN,NaN,NaN,10,CPF,842.939.188-68,CPF,842.939.188-68,True,False
4,PRZBB,JOSE CLAUDIO FURLAN,NaN,MG,45158940649,JOSE CLAUDIO FURLAN,NaN,MG,45158940649,111205,...,NaN,NaN,NaN,10,CPF,451.589.406-49,CPF,451.589.406-49,True,False


In [27]:
df = pd.DataFrame(r['data'])

In [28]:
df.head()

,Owner,Description,Discount,$currency_symbol,Engine_Model,$review_process,Tax,Invoice_Date,$review,$state,...,Product_Details,Subject,$orchestration,Engine_Serial_Number,$in_merge,$line_tax,Tag,Prop_Blades,$approval_state,Prop_Hub
0,"{'name': 'Rebecca Thorp', 'id': '5244140000028...",None,0,$,PT6A-67AG,"{'approve': False, 'reject': False, 'resubmit'...",0,2020-10-21,None,save,...,"[{'product': {'Product_Code': None, 'Currency'...",2020 AT-802 HDP Argentina,None,PCE-RD0355,False,[],[],"M09005, M09013, M09018, M09019, M09021",approved,HBA2452
1,"{'name': 'Rebecca Thorp', 'id': '5244140000028...",None,0,$,PT6A-15AG,"{'approve': False, 'reject': False, 'resubmit'...",0,2020-11-12,None,save,...,"[{'product': {'Product_Code': None, 'Currency'...",2020 AT-402B Aero Chaco,None,PCE-PD0268,False,[],[],"M09442, M09443, M09445",approved,BUA34635
2,"{'name': 'Rebecca Thorp', 'id': '5244140000028...",None,0,$,PT6A-11AG,"{'approve': False, 'reject': False, 'resubmit'...",0,2020-11-19,None,save,...,"[{'product': {'Product_Code': None, 'Currency'...",2020 AT-402A Medianeira,None,PCE-RH0139,False,[],[],"M09901, M09902, M09903",approved,BUA34651
3,"{'name': 'Rebecca Thorp', 'id': '5244140000028...",None,0,$,PT6A-34AG,"{'approve': False, 'reject': False, 'resubmit'...",0,2020-11-25,None,save,...,"[{'product': {'Product_Code': None, 'Currency'...",2020 AT-504 Aplimax,None,PCE-PH1348,False,[],[],"M09446, M09448, M09449",approved,BUA34632
4,"{'name': 'Rebecca Thorp', 'id': '5244140000028...",None,0,$,PT6A-34AG,"{'approve': False, 'reject': False, 'resubmit'...",0,2020-12-02,None,save,...,"[{'product': {'Product_Code': None, 'Currency'...",2020 AT-502B Agripar,None,PCE-PH1356,False,[],[],"M09440, M09441, M09433",approved,BUA34624


In [31]:
r['info']

{'per_page': 200, 'count': 200, 'page': 1, 'more_records': True}

In [32]:
r = crm.get("invoices")

127.0.0.1 - - [11/Feb/2021 15:22:57] "GET /?code=1000.791389c344dba9c8a6bd3b98b69a05ea.4d307d6bb9ec89c80720e77476218337&location=us&accounts-server=https%3A%2F%2Faccounts.zoho.com& HTTP/1.1" 200 -


In [33]:
r